In [2]:
import pandas as pd
# 여기서부터 시작 (결과)

dataname = 'Test-AMR_LDA-mixtral_instruct'
pred_foler = f'/hdd/hjl8708/workspace/experiments/{dataname}'
pred_file: str = f'{pred_foler}/greedy_preds.json'

predict_data = pd.read_json(pred_file)

predict_data

,qid,question,answer,answer_sentence,output
0,RULE_train_1348,<Passage>: Clark: Our local community theater ...,C,treats one main factor considered in the selec...,C\n\nClark's argument is
1,RULE_train_439,<Passage>: Electronic cigarettes should not be...,A,The current regulations are designed to preven...,(D)\n\nThe reason why (
2,RULE_train_1030,<Passage>: Politician: Most of those at the me...,A,It rejects an argument merely on the grounds t...,(A) It rejects an argument merely
3,RULE_train_314,<Passage>: Journalists agree universally that ...,B,Reporters disagree on what sort of behavior qu...,(B) Reporters disagree on what sort
4,RULE_train_79,<Passage>: Teachers should not do anything to ...,B,Students lose respect for teachers whenever th...,(B) \n\nThe passage suggests
...,...,...,...,...,...
4710,RULE_train_1231_optionD,<Passage>: Commentator: The quality of health ...,yes.,"Five years ago, the average first patient-phys...","No, this option does not weaken the"
4711,RULE_train_775_optionA,<Passage>: A recent study of perfect pitch -- ...,yes.,People who have relatives with perfect pitch g...,Yes.\n\nOption A strengthens the
4712,RULE_train_775_optionB,<Passage>: A recent study of perfect pitch -- ...,no.,All of the researchers conducting the study ha...,"No, option (B) does not strengthen"
4713,RULE_train_775_optionC,<Passage>: A recent study of perfect pitch -- ...,no.,People with perfect pitch are more likely than...,No.\n\nOption A is the correct


In [21]:
import string

def output_processing(text):
    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    
    cleaned_text = white_space_fix(remove_punc(lower(text)))
    return cleaned_text
    
def match_score(pred, target):
    pred = output_processing(pred)
    target = output_processing(target)
    
    max_len = len(target)
    
    return pred[:max_len] == target[:max_len]


def typing_row(answer):
    if output_processing(answer) in ['yes', 'no']:
        return 'option'
    else:
        return 'mcqa'

def option_selective(answer):
    if output_processing(answer) == 'yes':
        return 'selective'
    elif output_processing(answer) == 'no':
        return 'eliminative'
    
predict_data['result'] = predict_data.apply(lambda x: match_score(x['output'], x['answer']), axis=1)
predict_data['type'] = predict_data.apply(lambda x: typing_row(x['answer']), axis=1)
predict_data['option_type'] = predict_data.apply(lambda x: option_selective(x['answer']), axis=1)

In [25]:
# type이 mcqa인 row의 accuracy
mcqa_accuracy = predict_data[predict_data['type'] == 'mcqa']['result'].mean()                                     * 100
option_accuracy = predict_data[predict_data['type'] == 'option']['result'].mean()                                 * 100
option_selective_accuracy = predict_data[predict_data['option_type'] == 'selective']['result'].mean()             * 100
option_eliminative_accuracy = predict_data[predict_data['option_type'] == 'eliminative']['result'].mean()         * 100

print(f"""
      mcqa_accuracy: {mcqa_accuracy}
      opti_accuracy: {option_accuracy}
      opti_sele_accuracy: \t{option_selective_accuracy}
      opti_elim_accuracy: \t{option_eliminative_accuracy}
      """)


      mcqa_accuracy: 63.83881230116649
      opti_accuracy: 73.03817603393425
      opti_sele_accuracy: 	73.70095440084835
      opti_elim_accuracy: 	72.81724991162956
      
